# PyCon UK: Alzheimer's Disease Challenge Hackathon
### Create local train/test dataset


In [1]:
from dateutil import rrule
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from sklearn import metrics, model_selection, preprocessing, pipeline

import tadpole_metrics

%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
# Load TADPOLE data
DATA_DIR = Path('../data')
tadpole = pd.read_csv(DATA_DIR / 'TADPOLE_LB1_LB2.csv', low_memory=False)
tadpole.head()

RID        PTID VISCODE  SITE  D1  D2  LB1  LB2 COLPROT ORIGPROT  \
0    2  011_S_0002      bl    11   1   1    1    0   ADNI1    ADNI1   
1    3  011_S_0003      bl    11   1   0    1    0   ADNI1    ADNI1   
2    3  011_S_0003     m06    11   1   0    1    0   ADNI1    ADNI1   
3    3  011_S_0003     m12    11   1   0    1    0   ADNI1    ADNI1   
4    3  011_S_0003     m24    11   1   0    1    0   ADNI1    ADNI1   

     EXAMDATE DX_bl  DXCHANGE   AGE PTGENDER  PTEDUCAT         PTETHCAT  \
0  2005-09-08    CN       1.0  74.3     Male        16  Not Hisp/Latino   
1  2005-09-12    AD       3.0  81.3     Male        18  Not Hisp/Latino   
2  2006-03-13    AD       3.0  81.3     Male        18  Not Hisp/Latino   
3  2006-09-12    AD       3.0  81.3     Male        18  Not Hisp/Latino   
4  2007-09-12    AD       3.0  81.3     Male        18  Not Hisp/Latino   

  PTRACCAT  PTMARRY  APOE4      FDG  PIB  AV45  CDRSB  ADAS11  ADAS13  MMSE  \
0    White  Married    0.0  1.36926  NaN   NaN    0.0   10.67   18.67  28.0   
1    White  Married    1.0  1.09079  NaN   NaN    4.5   22.00   31.00  20.0   
2    White  Married    1.0  1.06360  NaN   NaN    6.0   19.00   30.00  24.0   
3    White  Married    1.0  1.10384  NaN   NaN    3.5   24.00   35.00  17.0   
4    White  Married    1.0  1.03871  NaN   NaN    8.0   25.67   37.67  19.0   

   RAVLT_immediate  RAVLT_learning  RAVLT_forgetting  RAVLT_perc_forgetting  \
0             44.0             4.0               6.0                54.5455   
1             22.0             1.0               4.0               100.0000   
2             19.0             2.0               6.0               100.0000   
3             31.0             2.0               7.0               100.0000   
4             23.0             1.0               5.0               100.0000   

    FAQ  MOCA  EcogPtMem  EcogPtLang  EcogPtVisspat  EcogPtPlan  EcogPtOrgan  \
0   0.0   NaN        NaN         NaN            NaN         NaN          NaN   
1  10.0   NaN        NaN         NaN            NaN         NaN          NaN   
2  12.0   NaN        NaN         NaN            NaN         NaN          NaN   
3  17.0   NaN        NaN         NaN            NaN         NaN          NaN   
4  14.0   NaN        NaN         NaN            NaN         NaN          NaN   

   EcogPtDivatt  EcogPtTotal  EcogSPMem  EcogSPLang  EcogSPVisspat  \
0           NaN          NaN        NaN         NaN            NaN   
1           NaN          NaN        NaN         NaN            NaN   
2           NaN          NaN        NaN         NaN            NaN   
3           NaN          NaN        NaN         NaN            NaN   
4           NaN          NaN        NaN         NaN            NaN   

   EcogSPPlan  EcogSPOrgan  EcogSPDivatt  EcogSPTotal      FLDSTRENG  \
0         NaN          NaN           NaN          NaN  1.5 Tesla MRI   
1         NaN          NaN           NaN          NaN  1.5 Tesla MRI   
2         NaN          NaN           NaN          NaN  1.5 Tesla MRI   
3         NaN          NaN           NaN          NaN  1.5 Tesla MRI   
4         NaN          NaN           NaN          NaN  1.5 Tesla MRI   

                                           FSVERSION  Ventricles  Hippocampus  \
0  Cross-Sectional FreeSurfer (FreeSurfer Version...    118233.0       8336.0   
1  Cross-Sectional FreeSurfer (FreeSurfer Version...     84599.0       5319.0   
2  Cross-Sectional FreeSurfer (FreeSurfer Version...     88580.0       5446.0   
3  Cross-Sectional FreeSurfer (FreeSurfer Version...     90099.0       5157.0   
4  Cross-Sectional FreeSurfer (FreeSurfer Version...     97420.0       5139.0   

   WholeBrain  Entorhinal  Fusiform  MidTemp        ICV        DX EXAMDATE_bl  \
0   1229740.0      4177.0   16559.0  27936.0  1984660.0        NL  2005-09-08   
1   1129830.0      1791.0   15506.0  18422.0  1920690.0  Dementia  2005-09-12   
2   1100060.0      2427.0   14400.0  16972.0  1906430.0  Dementia  2005-09-12   
3   1095640.0      1596.0   1

In [3]:
tadpole.columns[:10]

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'D1', 'D2', 'LB1', 'LB2', 'COLPROT',
       'ORIGPROT'],
      dtype='object')

In [4]:
outcomes = ["ADAS13", "DX", "Ventricles"]
cog_tests_attributes = ["CDRSB", "ADAS11", "MMSE", "RAVLT_immediate"]
mri_measures = ['Hippocampus', 'WholeBrain', 'Entorhinal', 'MidTemp' , "FDG", "AV45"]
pet_measures = ["FDG", "AV45"]
csf_measures = ["ABETA_UPENNBIOMK9_04_19_17", "TAU_UPENNBIOMK9_04_19_17", "PTAU_UPENNBIOMK9_04_19_17"]
risk_factors = ["APOE4", "AGE"]

In [5]:
# Add age at exam
tadpole.EXAMDATE = pd.to_datetime(tadpole.EXAMDATE)
tadpole_grouped = tadpole.groupby("RID").apply(lambda x:(x["EXAMDATE"]-x["EXAMDATE"].min()).dt.days/365.25 + x["AGE"].min())
tadpole_grouped.sort_index(inplace=True)
tadpole.sort_values(by=["RID", "EXAMDATE"], inplace=True)
tadpole["AGE_AT_EXAM"] = tadpole_grouped.values
tadpole['AGE_INT'] = tadpole['AGE_AT_EXAM'].apply(int)

# Create X, y datasets

In [6]:
dx_map = {
        'MCI': 'MCI',
        'NL': 'CN',
        'Dementia': 'AD',
        'MCI to Dementia': 'AD',
        'NL to MCI': 'MCI',
        'MCI to NL': 'CN',
        'Dementia to MCI': 'MCI',
        'NL to Dementia': 'AD'
    }
tadpole['diagnosis'] = tadpole['DX'].map(dx_map)

In [7]:
X_cols = [
    # cog_tests_attributes
    "CDRSB", "ADAS11", "MMSE", "RAVLT_immediate",
    # mri_measures
    'Hippocampus', 'WholeBrain', 'Entorhinal', 'MidTemp' , "FDG", "AV45",
    # pet_measures
    "FDG", "AV45",
    # csf_measures
    "ABETA_UPENNBIOMK9_04_19_17", "TAU_UPENNBIOMK9_04_19_17", "PTAU_UPENNBIOMK9_04_19_17",
    # risk_factors
    "APOE4", "AGE",
    # age
    'AGE_AT_EXAM',
]

X = tadpole[X_cols]
y_diag = tadpole['diagnosis']
y_adas = tadpole['ADAS13']
y_vent = tadpole['Ventricles']

# Create local train/test sets

In [8]:
# Instead let's train on all patients except last observation!
test_idx = []
patients_from_2016 = tadpole[tadpole['EXAMDATE'] >= pd.to_datetime('2016-01-01')]  # 363 of these
for rid in  patients_from_2016['RID'].unique():
    rid_df = tadpole[tadpole['RID'] == rid][['RID', 'ADAS13', 'Ventricles', 'diagnosis', 'EXAMDATE']].copy()
    rid_df.sort_values('EXAMDATE', ascending=True, inplace=True)
    if len(rid_df.dropna()) > 0:
        test_idx += [rid_df.dropna().index[-1]]

In [9]:
train_idx = list(set(tadpole.index) - set(test_idx))

In [10]:
# Create train/test dfs
train = tadpole.loc[train_idx]
test = tadpole.loc[test_idx]

In [11]:
# Cut off 2016 onwards in training data
train = train[train['EXAMDATE'] < pd.to_datetime('2016-01-01')].copy()
test = test[test['EXAMDATE'] >= pd.to_datetime('2016-01-01')].copy()

In [12]:
def month_year(x):
    return pd.to_datetime(x.strftime('%Y-%m-01'))

month_year(test['EXAMDATE'].min())

Timestamp('2016-01-01 00:00:00')

In [13]:
forecast_months = list(
    rrule.rrule(rrule.MONTHLY,
                dtstart=month_year(test['EXAMDATE'].min()),
                until=month_year(test['EXAMDATE'].max()))
)
forecast_months = [pd.to_datetime(x) for x in forecast_months]

In [14]:
# Create empty prediction dataframe
submission = pd.read_csv(DATA_DIR / 'TADPOLE_Submission_Pycon_Dummy.csv')
test_ids = test['RID'].unique()
preds_list = []
for rid in test_ids:
    preds_rid = submission.iloc[0:0].copy()
    preds_rid['Forecast Date'] = [x.strftime('%Y-%m') for x in forecast_months]
    preds_rid['RID'] = rid
    preds_rid['Forecast Month'] = range(1, (len(forecast_months) + 1))
    preds_list += [preds_rid]

preds = pd.concat(preds_list, axis=0)
preds.head()

,RID,Forecast Month,Forecast Date,CN relative probability,MCI relative probability,AD relative probability,ADAS13,ADAS13 50% CI lower,ADAS13 50% CI upper,Ventricles_ICV,Ventricles_ICV 50% CI lower,Ventricles_ICV 50% CI upper
0,4115,1,2016-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4115,2,2016-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4115,3,2016-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4176,1,2016-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4176,2,2016-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Basic model example

In [15]:
default_diagnosis = train['diagnosis'].dropna().value_counts() / len(train['diagnosis'].dropna())

In [16]:
default_ADAS13 = train['ADAS13'].dropna().mean()
default_ADAS13_CI_low = train['ADAS13'].dropna().quantile(.25)
default_ADAS13_CI_high = train['ADAS13'].dropna().quantile(.75)
default_ADAS13_CI_range = default_ADAS13_CI_high - default_ADAS13_CI_low

In [17]:
train['Ventricles_ICV'] = train['Ventricles'] / train['ICV']
test['Ventricles_ICV'] = test['Ventricles'] / test['ICV']
default_vent = train['Ventricles_ICV'].dropna().mean()
default_vent_CI_low = train['Ventricles_ICV'].dropna().quantile(.25)
default_vent_CI_high = train['Ventricles_ICV'].dropna().quantile(.75)
default_vent_CI_range = default_vent_CI_high - default_vent_CI_low

In [18]:
for rid in test_ids:
    train_rid = train[train['RID'] == rid]
    # diagnosis
    if len(train_rid['diagnosis'].dropna()) > 0:
        diagnosis = train_rid['diagnosis'].dropna().iloc[-1]
        preds.loc[preds['RID'] == rid, 'CN relative probability'] = int(diagnosis == 'CN')
        preds.loc[preds['RID'] == rid, 'MCI relative probability'] = int(diagnosis == 'MCI')
        preds.loc[preds['RID'] == rid, 'AD relative probability'] = int(diagnosis == 'AD')
    else:
        preds.loc[preds['RID'] == rid, 'CN relative probability'] = default_diagnosis['CN']
        preds.loc[preds['RID'] == rid, 'MCI relative probability'] = default_diagnosis['MCI']
        preds.loc[preds['RID'] == rid, 'AD relative probability'] = default_diagnosis['AD']
    # ADAS13
    if len(train_rid['ADAS13'].dropna()) > 0:
        adas13 = train_rid['ADAS13'].dropna().iloc[-1]
    else:
        adas13 = default_ADAS13
    preds.loc[preds['RID'] == rid, 'ADAS13'] = adas13
    preds.loc[preds['RID'] == rid, 'ADAS13 50% CI lower'] = adas13 - (default_vent_CI_range/2)
    preds.loc[preds['RID'] == rid, 'ADAS13 50% CI upper'] = adas13 + (default_vent_CI_range/2)
    # ventricles
    if len(train_rid['Ventricles_ICV'].dropna()) > 0:
        ventricles = train_rid['Ventricles_ICV'].dropna().iloc[-1]
    else:
        ventricles = default_vent
    preds.loc[preds['RID'] == rid, 'Ventricles_ICV'] = ventricles
    preds.loc[preds['RID'] == rid, 'Ventricles_ICV 50% CI lower'] = ventricles - (default_vent_CI_range/2)
    preds.loc[preds['RID'] == rid, 'Ventricles_ICV 50% CI upper'] = ventricles + (default_vent_CI_range/2)

# Compare preds vs test

In [19]:
# Filter preds on month_year combos in test
test['month_year'] = test['EXAMDATE'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m'))
test_preds = test[['RID', 'month_year']].merge(preds, how='left', left_on=['RID', 'month_year'], right_on=['RID', 'Forecast Date'])
test_preds.head()

,RID,month_year,Forecast Month,Forecast Date,CN relative probability,MCI relative probability,AD relative probability,ADAS13,ADAS13 50% CI lower,ADAS13 50% CI upper,Ventricles_ICV,Ventricles_ICV 50% CI lower,Ventricles_ICV 50% CI upper
0,4115,2016-02,2,2016-02,0.0,1.0,0.0,20.0,19.991427,20.008573,0.017040,0.008467,0.025613
1,4176,2016-01,1,2016-01,1.0,0.0,0.0,9.0,8.991427,9.008573,0.019991,0.011418,0.028564
2,4179,2016-01,1,2016-01,1.0,0.0,0.0,13.0,12.991427,13.008573,0.036064,0.027491,0.044638
3,4311,2016-01,1,2016-01,0.0,1.0,0.0,21.0,20.991427,21.008573,0.020659,0.012086,0.029232
4,4312,2016-01,1,2016-01,0.0,1.0,0.0,15.0,14.991427,15.008573,0.017707,0.009134,0.026280


In [20]:
# Spot check...

In [21]:
test[test['RID'] == 4115][['diagnosis', 'ADAS13', 'Ventricles_ICV', 'EXAMDATE']]

,diagnosis,ADAS13,Ventricles_ICV,EXAMDATE
11785,MCI,18.0,0.019155,2016-02-19


In [22]:
test_preds[test_preds['RID'] == 4115]

,RID,month_year,Forecast Month,Forecast Date,CN relative probability,MCI relative probability,AD relative probability,ADAS13,ADAS13 50% CI lower,ADAS13 50% CI upper,Ventricles_ICV,Ventricles_ICV 50% CI lower,Ventricles_ICV 50% CI upper
0,4115,2016-02,2,2016-02,0.0,1.0,0.0,20.0,19.991427,20.008573,0.01704,0.008467,0.025613


In [23]:
# Diagnosis MAUC
def evaluate_predictions(predictions, actuals):
    diagnosis_int_map = {'CN': 0, 'MCI': 1, 'AD': 2}

    # MAUC
    MAUC = tadpole_metrics.MAUC(list(zip(
        actuals['diagnosis'].map(diagnosis_int_map),
        predictions[['CN relative probability', 'MCI relative probability', 'AD relative probability']].values
    )))
    
    # BCA
    predicted_labels = [x.argmax() for x in predictions[
            ['CN relative probability', 'MCI relative probability', 'AD relative probability']
        ].values]
    BCA = tadpole_metrics.calcBCA(estimLabels=predicted_labels,
                                  trueLabels=actuals['diagnosis'].map(diagnosis_int_map),
                                  nrClasses=3)
    
    # ADAS MAE
    ADAS_MAE = metrics.mean_absolute_error(y_true=actuals['ADAS13'],
                                           y_pred=predictions['ADAS13'])
    
    # Ventricles MAE
    vent_MAE = metrics.mean_absolute_error(y_true=actuals['Ventricles_ICV'],
                                           y_pred=predictions['Ventricles_ICV'])
    
    # ADAS WES
    ADAS_WES = tadpole_metrics.calculate_WES(
        estimates=predictions['ADAS13'].values,
        lowers=predictions['ADAS13 50% CI lower'].values,
        uppers=predictions['ADAS13 50% CI upper'].values,
        trues=actuals['ADAS13'].values
        )
    
    # Vent WES
    vent_WES = tadpole_metrics.calculate_WES(
        estimates=predictions['Ventricles_ICV'].values,
        lowers=predictions['Ventricles_ICV 50% CI lower'].values,
        uppers=predictions['Ventricles_ICV 50% CI upper'].values,
        trues=actuals['Ventricles_ICV'].values
        )
    
    # ADAS_CPA
    ADAS_CPA = tadpole_metrics.calculate_CPA(
        estimates=predictions['ADAS13'].values,
        lowers=predictions['ADAS13 50% CI lower'].values,
        uppers=predictions['ADAS13 50% CI upper'].values,
        trues=actuals['ADAS13'].values
        )
    
    # Vent CPA
    vent_CPA = tadpole_metrics.calculate_CPA(
        estimates=predictions['Ventricles_ICV'].values,
        lowers=predictions['Ventricles_ICV 50% CI lower'].values,
        uppers=predictions['Ventricles_ICV 50% CI upper'].values,
        trues=actuals['Ventricles_ICV'].values
        )
    
    return {
        'MAUC': MAUC,
        'BCA': BCA,
        'ADAS_MAE': ADAS_MAE,
        'vent_MAE': vent_MAE,
        'ADAS_WES': ADAS_WES,
        'vent_WES': vent_WES,
        'ADAS_CPA': ADAS_CPA,
        'vent_CPA': vent_CPA
    }

In [24]:
evaluate_predictions(predictions=test_preds, actuals=test)

{'ADAS_CPA': 0.35714285714285715,
 'ADAS_MAE': 3.5714285714285716,
 'ADAS_WES': 3.5714285714285849,
 'BCA': 0.5,
 'MAUC': 0.9375,
 'vent_CPA': 0.4285714285714286,
 'vent_MAE': 0.0032750515091335914,
 'vent_WES': 0.0032750515091335935}